In [10]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.metrics import mean_squared_error
from binance.client import Client
from datetime import datetime
import userbinance
import talib
import time

In [11]:
def prepare_date(klines):
    klines = np.array(klines).astype(np.float64)[:, 0:6]
    RSI = talib.RSI(klines[:,4], timeperiod=14).round(2)
    EMA20 = talib.EMA(klines[:,4], timeperiod=20).round(2)
    EMA40 = talib.EMA(klines[:,4], timeperiod=40).round(2)
    EMA60 = talib.EMA(klines[:,4], timeperiod=60).round(2)

    fastk, fastd = talib.STOCHRSI(klines[:,4], timeperiod=14)
    fastk = fastk.round(2)
    fastd = fastd.round(2)

    macd_line, signal_line, _ = talib.MACD(klines[:,4])
    macd_line = macd_line.round(2)
    signal_line = signal_line.round(2)

    upper, middle,lower = talib.BBANDS(klines[:,4], timeperiod=5)
    upper = upper.round(2)
    middle = middle.round(2)
    lower = lower.round(2)

    adx_values = talib.ADX(klines[:,2], klines[:,3], klines[:,4], timeperiod=14).round(2)

    indicators = np.column_stack((RSI, EMA20, EMA40,EMA60,fastk,fastd,macd_line,signal_line,upper,middle,lower,adx_values))
    # Concatena
    klines = np.column_stack((klines, indicators))
    
    # elimina la primera columna
    klines = klines[-10:-1, 1:]
    return klines
    
    

In [12]:
# Modificar la definición del modelo LSTM para ser más complejo
class ComplexLSTM(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size, num_layers):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers

        # Capa LSTM con múltiples capas apiladas
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, batch_first=True).cuda()

        # Capa completamente conectada
        self.linear = nn.Linear(hidden_layer_size, output_size).cuda()

    def forward(self, input_seq):
        # Inicializar las celdas ocultas
        h0 = torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).cuda()
        c0 = torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).cuda()

        # Propagación hacia adelante
        lstm_out, _ = self.lstm(input_seq, (h0, c0))
        predictions = self.linear(lstm_out[:, -1, :])
        return predictions

In [13]:
# Verifica si tienes una GPU disponible y configura el dispositivo.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo actual:", device)

Dispositivo actual: cuda


In [14]:
interval = Client.KLINE_INTERVAL_4HOUR
client = Client(userbinance.API_KEY, userbinance.API_SECRET, tld='com')
symbolticker = 'BTCUSDT'

In [15]:
# Ruta donde guardaste el modelo entrenado
ruta_modelo_guardado = 'checkpoints/save_BTC_4hs.pth'
modelo = ComplexLSTM(input_size=16, hidden_layer_size=100, output_size=1, num_layers=4)
# Carga los pesos del modelo
checkpoint = torch.load(ruta_modelo_guardado)
modelo.load_state_dict(checkpoint['model_state_dict'])
# Coloca el modelo en modo de evaluación (importante para la inferencia)
modelo.eval()

ComplexLSTM(
  (lstm): LSTM(16, 100, num_layers=4, batch_first=True)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)

In [16]:
np.set_printoptions(threshold=np.inf)

In [17]:
klines = client.get_klines(symbol=symbolticker, interval=Client.KLINE_INTERVAL_4HOUR,limit=100)
klines =  prepare_date(klines)
new_data_tensor = torch.tensor(klines, dtype=torch.float32).to('cuda')
new_data_tensor.shape

torch.Size([9, 17])

***

In [18]:
while True:
    # Realiza una solicitud a la API de Binance para obtener datos del par de criptomonedas
    klines = client.get_klines(symbol=symbolticker, interval=Client.KLINE_INTERVAL_4HOUR,limit=100)
    klines =  prepare_date(klines)
        
    # Realiza una predicción con el modelo
    # Supongamos que tienes un nuevo punto de datos en tiempo real (por ejemplo, el precio actual)
    # Prepara el punto de datos de la misma manera que durante el entrenamiento
    # Asegúrate de tener todas las características necesarias
    new_data_tensor = torch.tensor(klines, dtype=torch.float32).to('cuda')
        
    # Realiza la predicción
    with torch.no_grad():
        prediccion = modelo(new_data_tensor)
        
    # Procesa la predicción según sea necesario
    valor_predicho = prediccion.item()
        
    # Imprime el precio actual y la predicción
    print(f'Predicción: {valor_predicho}')

    # Espera 1 segundo antes de realizar la siguiente solicitud
    time.sleep(1)


RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors